<a href="https://colab.research.google.com/github/amandatz/linear-programming/blob/main/Atividade3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 3



Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [18]:
using Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [19]:
using JuMP, HiGHS, LinearAlgebra, Printf

## Problema

A Pastesian é uma fábrica de massas familiar que está planejando a produção de lasanhas para os próximos 4 meses. Além do sabor tradicional, ela decidiu lançar um novo sabor de lasanha para esta temporada. A empresa planeja suas operações ao longo de 4 meses. Para o sabor tradicional, a demanda para os primeiros 3 meses é considerada conhecida com base em dados históricos: 200, 350 e 150 unidades, respectivamente.

A demanda do 4° mês para o sabor tradicional é mais incerta, pois coincide com a alta temporada na região e historicamente apresenta grande variabilidade. Para lidar com essa incerteza, Ricardo, o dono da Pastesian, trabalha com três cenários possíveis para a demanda do sabor tradicional no último mês, dados por:

- Cenário 1: 220 unidades  
- Cenário 2: 250 unidades  
- Cenário 3: 300 unidades  

Assume-se que esses cenários são equiprováveis, e o modelo de produção deve ser capaz de acomodar essas variações sem incorrer em custos excessivos de produção, estocagem ou falta de produto.

Paralelamente, o novo sabor de lasanha também precisa ser planejado. Estima-se que a demanda desse novo produto seja de 30, 70 e 140 unidades nos meses 1, 2 e 3, respectivamente. Para o 4° mês, a demanda pode assumir os seguintes valores, com as probabilidades esperadas indicadas a seguir:

| Demanda (unidades) | Probabilidade |
|--------------------|---------------|
| 200                | 0,3           |
| 240                | 0,5           |
| 300                | 0,2           |

Além disso, Ricardo pretende expandir gradualmente a infraestrutura de armazenamento ao longo da temporada. Dessa forma, a capacidade total de estoque da fábrica (somando lasanha tradicional e nova) em cada mês é limitada por:

| Mês | Capacidade de estoque (unidades) |
|-----|----------------------------------|
| 1   | 200                              |
| 2   | 220                              |
| 3   | 230                              |
| 4   | 250                              |

O objetivo é determinar, para cada mês e para cada sabor, quanto produzir e quanto manter em estoque, de forma a minimizar o custo total de operação (produção, estocagem e eventuais faltas), respeitando as limitações impostas.

## Parâmetros

In [20]:
# Períodos (mês)
T = 1:4;
# Custo (R$/unidade) de produção em cada mês
cp = [5.5, 7.2, 8.8, 10.9];
# Custo (R$/unidade) de manter as lasanhas em estoque, de um mês para o seguinte
ce = [1.3, 1.95, 2.2];
# Capacidade total de estoque em cada mês (para as duas lasanhas somadas)
E = [200, 220, 230, 250];

# Modelo para lasanha original

**Parâmetros**

In [21]:
# Demanda certa nos meses 1–3 (lasanha original)
D_o = [200, 350, 150];
# Lasanhas originais (unidade) em estoque inicialmente
e0_o = 50;

# Cenários para o mês 4
C_o = 1:3;
# Demanda incerta no mês 4
D4_o = [220, 250, 300]
# Penalidade por demanda não atendida no mês 4
pp_o = 5.045 # ONDE ENTRA ISSO????

5.045

**Modelo**

Este código constrói o modelo base contendo apenas os elementos referentes à lasanha original. Ele representa a parte fixa e comum aos dois problemas, tanto ao modelo determinístico quanto ao estocástico da lasanha nova, e, por isso, será reutilizado posteriormente na formulação de ambos.

Observe que a função objetivo ainda não é definida, pois ela difere entre os dois modelos finais. Aqui, o foco está apenas nas restrições estruturais da lasanha original. Em particular, consideramos:

- O estoque inicial do produto original: `e_o[1] = e0_o`;
- O balanço de estoque ao longo dos três primeiros meses: `e_o[t+1] = e_o[t] + l_o[t] - D_orig[t]` para os meses `t = 1,2,3`;
- A presença de incerteza na demanda do 4° mês, tratada por meio de variáveis específicas para cada cenário, já que não existe um único valor determinístico para esse período;
- A restrição de atendimento da demanda estocástica no mês 4, permitindo falta apenas através da variável de déficit `dp_o[c]`: `dp_o[c] >= D4_o[c] - (l4_o[c] + e_o[4])`;
- A restrição que garante que a produção no mês 4 seja única e idêntica para todos os cenários: `sum(l4_o) == length(C_o) * l4_o[c]`;



In [22]:
function build_base_original(model, T, D_o, e0_o, C, D4_o)
    ## ============================================================
    ## Variáveis
    ## ============================================================
    @variable(model, l_o[T] >= 0)       # produção da lasanha original
    @variable(model, e_o[T] >= 0)       # estoque da lasanha original
    @variable(model, l4_o[C] >= 0)      # produção no mês 4 por cenário
    @variable(model, dp_o[C] >= 0)      # demanda não atendida no mês 4 por cenário

    ## ============================================================
    ## Restrições (meses 1,2,3)
    ## ============================================================

    # Estoque inicial
    @constraint(model, e_o[1] == e0_o)

    # Balanço meses 1, 2, 3
    @constraint(model, BAL_o[t in T[1:end-1]],
        e_o[t+1] == e_o[t] + l_o[t] - D_o[t]
    )

    ## ============================================================
    ## Restrições (mês 4)
    ## ============================================================

    # Atender as demandas incertas no mês 4
    @constraint(model, DEM_o[c in C],
        dp_o[c] >= D4_o[c] - (l4_o[c] + e_o[4])
    )

    # A produção no mês 4 deve ser a mesma em todos os cenários
    @constraint(model, NA_o[c in C],
        sum(l4_o) == length(C) * l4_o[c]
    )

    return model, l_o, e_o, l4_o, dp_o
end

build_base_original (generic function with 1 method)

# Modelo determinístico para lasanha nova

**Parâmetros**

In [23]:
# Demanda certa nos meses 1–3 (lasanha nova)
D_n = [30, 70, 140];

# Cenários para o mês 4
C_o = 1:3;
# Demanda incerta no mês 4
D4_n = [200, 240, 300];
# Probabilidades
P4_n = [0.3, 0.5, 0.2];


In [24]:
# Demanda média esperada da lasanha nova no mês 4
d4_mean_n = dot(D4_n, P4_n);
D_n_det = vcat(D_n, d4_mean_n);

**Modelo**

In [25]:
model_det = Model(HiGHS.Optimizer);
model_det, l_o, e_o = build_base_original(model_det, T, D_o, e0_o, C_o, D4_o);

In [26]:
@variable(model_det, l_n[T] >= 0)   # produção da lasanha nova
@variable(model_det, e_n[T] >= 0)   # estoque da lasanha nova

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Vector{VariableRef}:
 e_n[1]
 e_n[2]
 e_n[3]
 e_n[4]

In [27]:
# Estoque inicial da lasanha nova é zero
@constraint(model_det, e_n[1] == 0);

# Balanço de estoque da lasanha nova (meses 1,2,3)
@constraint(model_det, BAL_n[t in T[1:end-1]],
    e_n[t+1] == e_n[t] + l_n[t] - D_n_det[t]
);

# Atendimento à demanda da lasanha nova (de fato só precisa em t=4,
# mas deixamos para todos os meses por simetria didática)
@constraint(model_det, AD_n[t in T],
    l_n[t] + e_n[t] >= D_n_det[t]
);

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 AD_n[1] : l_n[1] + e_n[1] ≥ 30
 AD_n[2] : l_n[2] + e_n[2] ≥ 70
 AD_n[3] : l_n[3] + e_n[3] ≥ 140
 AD_n[4] : l_n[4] + e_n[4] ≥ 240

In [28]:
# ============================
# Capacidade de estoque total
# ============================
@constraint(model_det, CE[t in T],
    e_o[t] + e_n[t] <= E[t]
);

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 CE[1] : e_o[1] + e_n[1] ≤ 200
 CE[2] : e_o[2] + e_n[2] ≤ 220
 CE[3] : e_o[3] + e_n[3] ≤ 230
 CE[4] : e_o[4] + e_n[4] ≤ 250

In [29]:
# Custo de produção + custo de estoque das duas lasanhas
@objective(model_det, Min,
    # Produção (original + nova)
    dot(cp, l_o) + dot(cp, l_n) +
    # Custo de estoque (mês 1->2, 2->3, 3->4) para original + nova
    dot(ce, e_o[2:end] .+ e_n[2:end])
);

5.5 l_o[1] + 7.2 l_o[2] + 8.8 l_o[3] + 10.9 l_o[4] + 5.5 l_n[1] + 7.2 l_n[2] + 8.8 l_n[3] + 10.9 l_n[4] + 1.3 e_o[2] + 1.3 e_n[2] + 1.95 e_o[3] + 1.95 e_n[3] + 2.2 e_o[4] + 2.2 e_n[4]

In [30]:
optimize!(model_det)

println("Status: ", termination_status(model_det))
println("FO ótima = ", objective_value(model_det))

println("\nProdução lasanha original (l_o): ", value.(l_o))
println("Estoque lasanha original (e_o):   ", value.(e_o))

println("\nProdução lasanha nova (l_n):      ", value.(l_n))
println("Estoque lasanha nova (e_n):        ", value.(e_n))

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 22 rows; 22 cols; 54 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 2e+00]
  Cost    [1e+00, 1e+01]
  Bound   [0e+00, 0e+00]
  RHS     [3e+01, 4e+02]
Presolving model
13 rows, 14 cols, 33 nonzeros  0s
Dependent equations search running on 4 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
10 rows, 11 cols, 24 nonzeros  0s
Presolve reductions: rows 10(-12); columns 11(-11); nonzeros 24(-30) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     9.9000000000e+02 Pr: 7(1160) 0s
          9     9.0940000000e+03 Pr: 0(0) 0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Simplex   iterations: 9
Objective value     :  9.0940000000e+03
P-D objective error :  0.0000000000e+00
Hi

### 1.5. Análise de sensibilidade das restrições de capacidade de estoque

Após resolver o modelo determinístico, realizei uma análise de sensibilidade sobre as restrições de capacidade total de estoque, isto é, sobre as desigualdades

$$
  e_o(t) + e_n(t) \leq E_t, \qquad t = 1,2,3,4.
$$

Como essas restrições acoplam diretamente os estoques da lasanha tradicional e da nova, elas são cruciais para entender até que ponto a solução ótima obtida permanece válida quando modificamos os valores de capacidade $E_t$.

Para cada mês $t = 1,2,3,4$, verifiquei:

- o **intervalo de variação permitido para $E_t$** (para mais e para menos);
- se a **base ótima permanece inalterada** dentro desse intervalo — isto é, se a solução ótima de produção e estoque para os dois sabores continua sendo a mesma;
- e o que acontece quando ultrapassamos esse intervalo (por exemplo, se outra restrição passa a se tornar ativa, se o padrão de produção muda, ou se o custo total começa a variar).

Essa análise permite entender quão "sensível" é a solução ótima às capacidades de armazenamento em cada mês e fornece ao Ricardo uma indicação de até onde ele pode expandir ou reduzir o espaço de estoque sem alterar o plano ótimo previamente calculado.


In [33]:
report = lp_sensitivity_report(model_det);

SensitivityReport{Float64}(Dict{ConstraintRef, Tuple{Float64, Float64}}(dp_o[1] ≥ 0 => (-Inf, 220.0), e_n[1] ≥ 0 => (-Inf, -0.0), e_o[1] = 50 => (-50.0, 150.0), AD_n[3] : l_n[3] + e_n[3] ≥ 140 => (0.0, 240.0), e_o[1] ≥ 0 => (-Inf, 50.0), AD_n[4] : l_n[4] + e_n[4] ≥ 240 => (-240.0, Inf), CE[2] : e_o[2] + e_n[2] ≤ 220 => (-150.0, 200.0), dp_o[3] ≥ 0 => (-Inf, 300.0), e_n[1] = 0 => (0.0, 100.0), BAL_n[2] : -l_n[2] - e_n[2] + e_n[3] = -70 => (-150.0, -0.0), DEM_o[3] : e_o[4] + l4_o[3] + dp_o[3] ≥ 300 => (-300.0, Inf), l_o[1] ≥ 0 => (-Inf, 300.0), CE[1] : e_o[1] + e_n[1] ≤ 200 => (-150.0, Inf), e_n[4] ≥ 0 => (-Inf, -0.0), l_o[2] ≥ 0 => (-Inf, 200.0), e_o[2] ≥ 0 => (-Inf, 150.0), NA_o[3] : l4_o[1] + l4_o[2] - 2 l4_o[3] = 0 => (0.0, -0.0), l_o[3] ≥ 0 => (-Inf, 150.0), BAL_o[2] : -l_o[2] - e_o[2] + e_o[3] = -350 => (-Inf, 200.0), BAL_o[3] : -l_o[3] - e_o[3] + e_o[4] = -150 => (-Inf, 150.0), l4_o[2] ≥ 0 => (-Inf, -0.0), DEM_o[2] : e_o[4] + l4_o[2] + dp_o[2] ≥ 250 => (-250.0, Inf), l4_o[1] ≥ 0 =

# Modelo estocástico para lasanha nova

In [31]:
model_estocastico = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none